# Building simulation for NET 02:
## 2x2 superblocks
This notebook summarizes the simulation construction and running for net 02

In [1]:
# Defining network file
NET_FILE = ".\data/grid01-bcn_10.net.xml"

### Demand generation
Endogenous demand based on ACTIVITYGEN

In [2]:
## Config file for parsing demographic data:

ACTIVITYGEN_STAT_FILE = ".\data/activitygen-bcn_10.stat.xml"

## Basic trip output file (without routing) as a resukt of the demand generation based on ACTIVITYGEN.
TRIPS_FROM_ACTIVITYGEN_STAT = ".\data/activitygen-bcn_10.trips.rou.xml"

## Routes output files from ACTIVITYGEN trips
ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN = ".\data/activitygen-bcn_10.rou.xml"

In [ ]:
# Calling ACTIVITYGEN tool in SUMO
# Random seed set for reproductibility purposes

!activitygen --net-file {NET_FILE} \
            --stat-file {ACTIVITYGEN_STAT_FILE} \
            --output-file {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --seed 25101987

In [ ]:
# Routing trips from ACTIVITYGEN
# CH routing algorithm is used

!duarouter --net-file {NET_FILE} \
            --route-files {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --output-file {ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN} \
            --routing-algorithm CH \
            --ignore-errors

Passing through traffic from random trips between fringe edges

In [ ]:
# creation of hourly distribution of trips
dist = [211, 218, 257, 467, 1058, 2094, 3766, 5537, 6722, 6830, 5759, 4798, 4360, 4711, 4982, 5541, 6175, 6602, 6449, 5346, 3966, 2633, 1807, 1199]
total_random_trips = 297500 #+17500 trips for compensating less activity gen trips due to defaults in edge assignation
factor = total_random_trips/sum(dist)
random_dist = [int(element * factor) for element in dist]

In [ ]:
# Creating iteratively random trips accordingly to the hourly demand
## CREATE RANDOM FOLDER if it does not exist
import os
if not os.path.exists('data/random'):
    os.makedirs('data/random')

for i in range(len(random_dist)):
    begin = 3600*i
    end = 3600*(i+1)
    trips = random_dist[i]
    period = (end-begin)/trips
    trip_prefix = str(str(i) + "h")
    output_folder_1 = "data/random/random_h_"
    output_folder_2_trips = "-bcn_10.trips.rou.xml"
    output_folder_2_routes = "-bcn_10.rou.xml"

    BEGIN_ARG = str("-b " + str(begin))
    END_ARG = str("-e " + str(end))
    PERIOD_ARG = str("-p" + str(period))
    OUT_TRIPS_ARG = str("-o" + output_folder_1 + str(i) + output_folder_2_trips)
    OUT_ROUTES_ARG = str("-r" + output_folder_1 + str(i) + output_folder_2_routes)
    
    # bash SUMO command for RANDOM trips
    # Adjust the installation path for SUMO
    !python "C:\Program Files (x86)\Eclipse\Sumo\tools\randomTrips.py" -n {NET_FILE} \
    --prefix={trip_prefix} \
    --fringe-factor 100 \
    {BEGIN_ARG} \
    {END_ARG} \
    {PERIOD_ARG} \
    --binomial 10 \
    "{OUT_TRIPS_ARG}" \
    --edge-permission=passenger

In [ ]:
# Merging all random routing files i|n a single file
import os
list_trips_files = []
for file in os.listdir(r"data/random/"):
    if file.endswith(".trips.rou.xml"):
        list_trips_files.append(str("data/random/" + file))
list_trips_files_str = ' '.join([str(str(elem) + ',') for elem in list_trips_files])
list_trips_files_str = list_trips_files_str.replace(" ","")

SINGLE_RANDOM_ROUTES_FILE = "data/random/random_ALL.rou.xml"

!duarouter --net-file {NET_FILE} \
--route-files {list_trips_files_str[0:-1]} \
--output-file {SINGLE_RANDOM_ROUTES_FILE} \
--routing-algorithm CH \
--ignore-errors

### Microscopic Simulation
#### Setting configuration files
A XML configuration file (**data/compar_032.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file.xml**) for results colletion.

In [3]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder
import os, time
if not os.path.exists('data/results'):
    os.makedirs('data/results')

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output data/results/overall_stats.xml'.format(scale)
    SUMMARY_ARG = '--summary data/results/summary.xml'.format(scale)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_032.sumo.cfg \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1 \
    -W
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

STARTING sumo simulation with scale 2.00
Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 30 ACT 30 BUF 2)                               
Step #200.00 (1ms ~= 1000.00*RT, ~55000.00UPS, vehicles TOT 61 ACT 55 BUF 1)              
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 100 ACT 66 BUF 0)                              
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 133 ACT 66 BUF 1)                              
Step #500.00 (1ms ~= 1000.00*RT, ~73000.00UPS, vehicles TOT 174 ACT 73 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~70000.00UPS, vehicles TOT 204 ACT 70 BUF 0)             
Step #700.00 (1ms ~= 1000.00*RT, ~56000.00UPS, vehicles TOT 229 ACT 56 BUF 3)             
Step #800.00 (1ms ~= 1000.00*RT, ~67000.00UPS, vehicles TOT 268 ACT 67 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~74000.00UPS, vehicles TOT 302 ACT 74 BUF 0)             
Step #1000.00 (1m

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (1ms ~= 1000.00*RT, ~29000.00UPS, vehicles TOT 29 ACT 29 BUF 2)              
Step #200.00 (1ms ~= 1000.00*RT, ~54000.00UPS, vehicles TOT 59 ACT 54 BUF 1)              
Step #300.00 (2ms ~= 500.00*RT, ~33000.00UPS, vehicles TOT 97 ACT 66 BUF 0)               
Step #400.00 (1ms ~= 1000.00*RT, ~63000.00UPS, vehicles TOT 129 ACT 63 BUF 1)             
Step #500.00 (1ms ~= 1000.00*RT, ~72000.00UPS, vehicles TOT 169 ACT 72 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~72000.00UPS, vehicles TOT 198 ACT 72 BUF 0)             
Step #700.00 (1ms ~= 1000.00*RT, ~52000.00UPS, vehicles TOT 223 ACT 52 BUF 3)             
Step #800.00 (1ms ~= 1000.00*RT, ~64000.00UPS, vehicles TOT 261 ACT 64 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~74000.00UPS, vehicles TOT 294 ACT 74 BUF 0)             
Step #1000.00 (1ms ~= 1000.00*RT, ~72000.00UPS, vehicles T

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (1ms ~= 1000.00*RT, ~28000.00UPS, vehicles TOT 28 ACT 28 BUF 2)              
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 58 ACT 53 BUF 0)                               
Step #300.00 (1ms ~= 1000.00*RT, ~64000.00UPS, vehicles TOT 95 ACT 64 BUF 0)              
Step #400.00 (1ms ~= 1000.00*RT, ~63000.00UPS, vehicles TOT 126 ACT 63 BUF 1)             
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 165 ACT 71 BUF 0)                              
Step #600.00 (1ms ~= 1000.00*RT, ~66000.00UPS, vehicles TOT 193 ACT 66 BUF 0)             
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 217 ACT 51 BUF 3)                              
Step #800.00 (1ms ~= 1000.00*RT, ~61000.00UPS, vehicles TOT 254 ACT 61 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~69000.00UPS, vehicles TOT 286 ACT 69 BUF 0)             
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 315 ACT 68 BUF

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (1ms ~= 1000.00*RT, ~27000.00UPS, vehicles TOT 27 ACT 27 BUF 2)              
Step #200.00 (1ms ~= 1000.00*RT, ~50000.00UPS, vehicles TOT 56 ACT 50 BUF 1)              
Step #300.00 (1ms ~= 1000.00*RT, ~63000.00UPS, vehicles TOT 92 ACT 63 BUF 0)              
Step #400.00 (1ms ~= 1000.00*RT, ~62000.00UPS, vehicles TOT 123 ACT 62 BUF 0)             
Step #500.00 (1ms ~= 1000.00*RT, ~67000.00UPS, vehicles TOT 160 ACT 67 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~64000.00UPS, vehicles TOT 188 ACT 64 BUF 0)             
Step #700.00 (1ms ~= 1000.00*RT, ~51000.00UPS, vehicles TOT 212 ACT 51 BUF 2)             
Step #800.00 (1ms ~= 1000.00*RT, ~60000.00UPS, vehicles TOT 247 ACT 60 BUF 0)             
Step #900.00 (2ms ~= 500.00*RT, ~34000.00UPS, vehicles TOT 279 ACT 68 BUF 0)              
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 307 ACT 68 BUF


Step #24300.00 (48ms ~= 20.83*RT, ~70729.17UPS, vehicles TOT 21543 ACT 3395 BUF 25724)    
Step #24305.00 (49ms ~= 20.41*RT, ~69285.71UPS, vehicles TOT 21543 ACT 3395 BUF 25748)    

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 27 ACT 27 BUF 1)                               
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 55 ACT 50 BUF 0)                               
Step #300.00 (1ms ~= 1000.00*RT, ~60000.00UPS, vehicles TOT 90 ACT 60 BUF 0)              
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 120 ACT 61 BUF 0)                              
Step #500.00 (1ms ~= 1000.00*RT, ~66000.00UPS, vehicles TOT 156 ACT 66 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~65000.00UPS, vehicles TOT 183 ACT 65 BUF 0)             
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 206 ACT 51 BUF 2)                              
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 241 ACT 63 BU

...

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 4 ACT 4 BUF 0)                                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 4 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 3 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 9 ACT 3 BUF 0)                                 
Step #600.00 (1ms ~= 1000.00*RT, ~3000.00UPS, vehicles TOT 11 ACT 3 BUF 0)                
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 12 ACT 3 BUF 0)                                
Step #800.00 (1ms ~= 1000.00*RT, ~4000.00UPS, vehicles TOT 14 ACT 4 BUF 0)                
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 15 ACT 4 BUF 0)                                
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 17 ACT 3 BUF 0

Step #26600.00 (1ms ~= 1000.00*RT, ~132000.00UPS, vehicles TOT 3688 ACT 132 BUF 1)        
Step #26700.00 (2ms ~= 500.00*RT, ~64500.00UPS, vehicles TOT 3745 ACT 129 BUF 0)          
Step #26800.00 (1ms ~= 1000.00*RT, ~126000.00UPS, vehicles TOT 3803 ACT 126 BUF 0)        
Step #26900.00 (1ms ~= 1000.00*RT, ~128000.00UPS, vehicles TOT 3859 ACT 128 BUF 0)        
Step #27000.00 (1ms ~= 1000.00*RT, ~129000.00UPS, vehicles TOT 3913 ACT 129 BUF 0)        
Step #27100.00 (1ms ~= 1000.00*RT, ~138000.00UPS, vehicles TOT 3972 ACT 138 BUF 0)        
Step #27200.00 (1ms ~= 1000.00*RT, ~126000.00UPS, vehicles TOT 4027 ACT 126 BUF 0)        
Step #27300.00 (1ms ~= 1000.00*RT, ~124000.00UPS, vehicles TOT 4085 ACT 124 BUF 0)        
Step #27400.00 (1ms ~= 1000.00*RT, ~132000.00UPS, vehicles TOT 4142 ACT 132 BUF 0)        
Step #27500.00 (2ms ~= 500.00*RT, ~68000.00UPS, vehicles TOT 4199 ACT 136 BUF 0)          
Step #27600.00 (2ms ~= 500.00*RT, ~70500.00UPS, vehicles TOT 4257 ACT 141 BUF 0)          

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 2 ACT 2 BUF 0)                                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 2 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 4 ACT 1 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 2 BUF 0)                                 
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 6 ACT 2 BUF 0)                                 
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 6 ACT 2 BUF 0)                                 
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 2 BUF 0)                                 
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 8 ACT 2 BUF 0)                                 
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 9 ACT 2 BUF 0)

Step #13100.00 (0ms ?*RT. ?UPS, vehicles TOT 162 ACT 5 BUF 0)                             
Step #13200.00 (0ms ?*RT. ?UPS, vehicles TOT 163 ACT 2 BUF 0)                             
Step #13300.00 (0ms ?*RT. ?UPS, vehicles TOT 165 ACT 4 BUF 0)                             
Step #13400.00 (0ms ?*RT. ?UPS, vehicles TOT 167 ACT 4 BUF 0)                             
Step #13500.00 (1ms ~= 1000.00*RT, ~6000.00UPS, vehicles TOT 170 ACT 6 BUF 0)             
Step #13600.00 (0ms ?*RT. ?UPS, vehicles TOT 172 ACT 4 BUF 0)                             
Step #13700.00 (0ms ?*RT. ?UPS, vehicles TOT 174 ACT 6 BUF 0)                             
Step #13800.00 (0ms ?*RT. ?UPS, vehicles TOT 176 ACT 3 BUF 0)                             
Step #13900.00 (1ms ~= 1000.00*RT, ~4000.00UPS, vehicles TOT 179 ACT 4 BUF 0)             
Step #14000.00 (0ms ?*RT. ?UPS, vehicles TOT 181 ACT 4 BUF 0)                             
Step #14100.00 (0ms ?*RT. ?UPS, vehicles TOT 183 ACT 5 BUF 0)                             

Step #72600.00 (1ms ~= 1000.00*RT, ~41000.00UPS, vehicles TOT 14278 ACT 41 BUF 0)         
Step #72700.00 (0ms ?*RT. ?UPS, vehicles TOT 14296 ACT 43 BUF 0)                          
Step #72800.00 (1ms ~= 1000.00*RT, ~43000.00UPS, vehicles TOT 14315 ACT 43 BUF 0)         
Step #72900.00 (1ms ~= 1000.00*RT, ~45000.00UPS, vehicles TOT 14332 ACT 45 BUF 0)         
Step #73000.00 (0ms ?*RT. ?UPS, vehicles TOT 14349 ACT 42 BUF 0)                          
Step #73100.00 (1ms ~= 1000.00*RT, ~37000.00UPS, vehicles TOT 14368 ACT 37 BUF 0)         
Step #73200.00 (1ms ~= 1000.00*RT, ~37000.00UPS, vehicles TOT 14386 ACT 37 BUF 0)         
Step #73300.00 (0ms ?*RT. ?UPS, vehicles TOT 14404 ACT 34 BUF 0)                          
Step #73400.00 (1ms ~= 1000.00*RT, ~38000.00UPS, vehicles TOT 14424 ACT 38 BUF 0)         
Step #73500.00 (1ms ~= 1000.00*RT, ~35000.00UPS, vehicles TOT 14441 ACT 35 BUF 0)         
Step #73600.00 (1ms ~= 1000.00*RT, ~37000.00UPS, vehicles TOT 14458 ACT 37 BUF 0)         

### Mesoscopic Simulations
#### Setting configuration files
A XML configuration file (**data/compar_032_meso.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file_meso.xml**) for results collection.

Three different calibrations are finally run for the mesoscopic case, by changing the headway times ($\tau values$) and other mesoscopic parameters as follows

#### Mesoscopic simulation trial01

Param:

$\tau_{ff} = 1$

$\tau_{fj} = 1.2$

$\tau_{jf} = 1.8$

$\tau_{jj} = 1.4$

--meso-tls-penalty = 2

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_01 = r'data/results_meso_trial01'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_01):
    os.makedirs(RESULTS_FOLDER_MESO_01)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_01)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_01)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_032_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 1 \
    --meso-taufj 1.2 \
    --meso-taujf 1.8 \
    --meso-taujj 1.4 \
    --meso-lane-queue \
    --meso-tls-penalty 2 \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial02

Param:

$\tau_{ff} = 2.1$

$\tau_{fj} = 0.75$

$\tau_{jf} = 0.75$

$\tau_{jj} = 0.5$

--meso-jam-threshold = -0.5

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_02 = r'data/results_meso_trial02'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_02):
    os.makedirs(RESULTS_FOLDER_MESO_02)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_02)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_02)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_032_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 2.1 \
    --meso-taufj 0.75 \
    --meso-taujf 0.75 \
    --meso-taujj 0.5 \
    --meso-lane-queue \
    --meso-jam-threshold -0.5 \
    --meso-junction-control \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial03

Params:

$\tau_{ff} = 1.13$

$\tau_{fj} = 1.13$

$\tau_{jf} = 1.73$

$\tau_{jj} = 1.4$

--meso-jam-threshold = -1

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_03 = r'data/results_meso_trial03'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_03):
    os.makedirs(RESULTS_FOLDER_MESO_03)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_03)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_03)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_032_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-junction-control \
    --meso-lane-queue \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value